# 펭귄 종 분류: 데이터 전처리 및 로지스틱 회귀 모델 학습

이 노트북은 펭귄 데이터셋을 사용하여 펭귄의 종(species)을 분류하는 과정을 다룹니다. 특히, 데이터 전처리 단계에서 **범주형 특성(categorical features)을 수동으로 라벨 인코딩하고, 결측치(missing values)를 처리하는 방법**에 초점을 맞춰 구성되었습니다. 최종적으로 로지스틱 회귀 모델을 학습시키고 그 성능을 평가합니다.

### 1. 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### 2. 데이터 로드 및 초기 탐색

`penguins.csv` 파일을 불러와 데이터의 기본적인 구조와 내용을 확인합니다.

In [ ]:
try:
    df = pd.read_csv("./data/penguins.csv")
    print("데이터 로드 성공!")
except FileNotFoundError:
    print("오류: './data/penguins.csv' 파일을 찾을 수 없습니다.")
    print("노트북이 있는 폴더(02_practice/0716) 아래에 'data/penguins.csv' 파일을 위치시켜 주세요.")
    df = None

if df is not None:
    print("
데이터 상위 5행:")
    print(df.head())
    print("
컬럼 정보:")
    print(df.columns)
    print("
데이터 통계 요약:")
    print(df.describe())
    print("
데이터 정보 (결측치 포함):")
    df.info()
    print("
결측치 개수:")
    print(df.isna().sum())

### 3. 범주형 특성 인코딩 (수동 라벨 인코딩)

`island`와 `sex` 컬럼은 문자열 형태의 범주형 데이터이므로, 머신러닝 모델이 처리할 수 있도록 숫자 형태로 변환해야 합니다. 여기서는 `df.loc`를 사용하여 수동으로 라벨 인코딩을 수행합니다.

**참고**: `scikit-learn`의 `LabelEncoder`나 `pandas`의 `get_dummies`와 같은 함수를 사용하면 더 자동화되고 견고하게 범주형 데이터를 처리할 수 있습니다.

In [ ]:
if df is not None:
    # 'island' 컬럼 인코딩
    df.loc[df["island"] == "Torgersen", "island"] = 1
    df.loc[df["island"] == "Dream",     "island"] = 2
    df.loc[df["island"] == "Biscoe",    "island"] = 3

    # 'sex' 컬럼 인코딩
    df.loc[df["sex"] == "MALE", "sex"] = 1
    df.loc[df["sex"] == "FEMALE", "sex"] = 2

    print("
인코딩 후 데이터 상위 10행:")
    print(df.head(10))
    print("
인코딩 후 데이터 타입:")
    print(df.dtypes)

### 4. 결측치 처리

데이터에 남아있는 결측치(`NaN`)를 확인하고, `dropna()` 함수를 사용하여 결측치가 포함된 행을 제거합니다.

In [ ]:
if df is not None:
    print("
결측치 개수 (처리 전):")
    print(df.isna().sum())

    # 결측치가 있는 행 전체 삭제
    df = df.dropna(how="any", axis=0) 

    print("
결측치 개수 (처리 후):")
    print(df.isna().sum())
    print("
데이터 형태 (결측치 제거 후):")
    print(df.shape)

### 5. 특성(X)과 타겟(y) 분리

모델 학습을 위해 입력 특성(`X`)과 예측 대상인 타겟(`y`)을 분리합니다. 펭귄 종(`species`)이 타겟 변수입니다.

In [ ]:
if df is not None:
    # 첫 번째 컬럼(species)을 타겟(y)으로, 나머지를 특성(X)으로 사용
    X = df.iloc[:, 1:] 
    y = df.iloc[:, 0] 

    print("
특성 데이터 (X) 상위 4행:")
    print(X.head(4))
    print("
타겟 데이터 (y) 상위 4행:")
    print(y.head(4))

### 6. 데이터 분할

모델의 일반화 성능을 평가하기 위해 데이터를 훈련 세트와 테스트 세트로 분할합니다.

In [ ]:
if df is not None:
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0 )

    print(f"훈련 데이터 형태: {X_train.shape}, {y_train.shape}")
    print(f"테스트 데이터 형태: {X_test.shape}, {y_test.shape}")

### 7. 모델 학습 및 평가 (로지스틱 회귀)

로지스틱 회귀 모델을 사용하여 펭귄 종을 분류하고, 훈련 세트와 테스트 세트에 대한 정확도를 평가합니다.

**참고**: 로지스틱 회귀와 같은 거리 기반 모델은 특성 스케일링(예: `StandardScaler`, `MinMaxScaler`)에 민감합니다. 이 노트북에서는 스케일링을 적용하지 않았지만, 실제 프로젝트에서는 스케일링을 통해 모델 성능을 더욱 향상시킬 수 있습니다.

In [ ]:
if df is not None:
    model = LogisticRegression(max_iter=1000) # 수렴을 위해 max_iter 증가
    model.fit(X_train, y_train)

    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

    print(f"훈련 세트 정확도: {train_score:.4f}")
    print(f"테스트 세트 정확도: {test_score:.4f}")

### 결론

이 노트북에서는 펭귄 데이터셋을 사용하여 범주형 특성을 수동으로 인코딩하고 결측치를 제거한 후, 로지스틱 회귀 모델을 학습시켜 펭귄 종을 분류하는 과정을 살펴보았습니다. 데이터 전처리 과정은 모델의 성능에 큰 영향을 미치므로, 데이터의 특성에 맞는 적절한 전처리 기법을 선택하는 것이 중요합니다.